In [48]:
import os
import numpy as np
from collections import Counter

# Import all other necessary libraries. Your code below ...
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score


In [49]:
#define function for creating list of words
def make_Dictionary(file_loc):
  word_total = []    #empty list to contain all words from emails
  emails = [os.path.join(file_loc,f) for f in os.listdir(file_loc)]    #use list comprehension to retrieve all emails in directory
    #Open each email and run through loop
  for email in emails:
    with open(email) as e:      #use with to ensure each email is opened and then closed
      for line in e:
        words = line.split()          #split words on at white space
        word_total += words          #add new words to word total
  dictionary = Counter(word_total)          #use Counter to count how many times each word appears
  list_to_remove = list(dictionary)     #Cleaning list of words

  for item in list_to_remove:
    if item.isalpha() == False:       #remove nonalphabet words
      del dictionary[item]
    elif len(item) == 1:         #remove any characters with a length of 1
      del dictionary[item]
  dictionary = dictionary.most_common(3000)         #only keep the 3000 most common words
    #return dictionary
  return dictionary

In [50]:
#define function for extracting words from emails and assigning them to spam or not
def extract_features(email_dir):
  files = [os.path.join(email_dir,ed) for ed in os.listdir(email_dir)]     #create path to email files
  features_matrix = np.zeros((len(files),3000)) #create feature matrix with a length of 3000 given our dictionary is limited to the top 3000 words
  train_labels = np.zeros(len(files)) #train labels allows us to assign a 0 or a 1 or each email indicating if the email is spam or not
  count = 1;    #count of spam emails?
  docID = 0;     #docID used to keep track of what document the loop is on in the list
    #loop over all files
  for file in files:
    with open(file) as fi:
        #go through each line of the file
      for i, line in enumerate(fi):
        if i ==2:          #stop at line 3
          words = line.split()             #split words on whitespace on line 3
            #look at each word collected
          for word in words:               #reset word_score to 0 for each new loop
            word_score = 0               #try to map each word to a word in the dictionary
            for i, d in enumerate(dictionary):
              if d[0] == word: #if match of word is found in dictionary then set that words score to the corrosponding in the dictionary
                word_score = i
                features_matrix[docID,word_score] = words.count(word)
      train_labels[docID] = 0; #set default state for each email as not spam
      filepathTokens = file.split('/') #extract part of path
      lastToken = filepathTokens[len(filepathTokens)-1] #extract last part of file name
      if lastToken.startswith("spmsg"): #check if file name starts with "spmsg" if so set email to spam
        train_labels[docID] = 1; #updates document with completed file
        count = count + 1 #both increment the counts of the loop
      docID = docID + 1
  return features_matrix, train_labels     #return the completed features_matrix and train_labels        

In [51]:
# Enter the "path" of your "train_mails" and "test-mails" FOLDERS in this cell ...
TRAIN_DIR = 'Data/train-mails'
TEST_DIR = 'Data/test-mails'

In [52]:
dictionary = make_Dictionary(TRAIN_DIR) #calls make_Dictionary function of TRAIN_DIR : collects 3000 most common words and their frequency

print ("reading and processing emails from TRAIN and TEST folders")
#calls extract_features on both train and test data sets
features_matrix, labels = extract_features(TRAIN_DIR) 
test_features_matrix, test_labels = extract_features(TEST_DIR)
#return is each email as vector of word counts (features_matrix) and label of spam or not spam

reading and processing emails from TRAIN and TEST folders


In [53]:
def load_emails(directory): #load emails and place them into empty lists with the emails and labels for the emails
    emails, labels = [], []
    for filename in os.listdir(directory): #retrieve file names
        filepath = os.path.join(directory, filename) #get full path to files
        with open(filepath, "r", encoding="latin1") as file:
            content = file.read() 
            emails.append(content) #email is read and appended to the the content list
            labels.append(1 if filename.startswith("spmsg") else 0)  # 1 for spam, 0 for ham
    return emails, np.array(labels) # return emails and labels

In [54]:
# Load training and test data
train_emails, train_labels = load_emails(TRAIN_DIR)
test_emails, test_labels = load_emails(TEST_DIR)

In [55]:
# Convert text into numerical features using TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=3000) #max features at 3000 for our limit
# Apply the vectorizer to the training emails and store the features
train_features = vectorizer.fit_transform(train_emails).toarray()
# Apply the vectorizer to the training emails and store the features
test_features = vectorizer.transform(test_emails).toarray()

In [56]:
# Train Naive Bayes model
print("Training Model using Gaussian Naive Bayes algorithm...")
model = GaussianNB() #better for normally distributed data
model.fit(train_features, train_labels) #fit with test and train labels
print("Training completed.")

Training Model using Gaussian Naive Bayes algorithm...
Training completed.


In [57]:
#Predict labels for test data
print("Testing trained model to predict Test Data labels...")
predicted_labels = model.predict(test_features)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predicted_labels)
print("Completed classification of the Test Data.")
print("Now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print(accuracy)

Testing trained model to predict Test Data labels...
Completed classification of the Test Data.
Now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9461538461538461


In [58]:
# Train Naive Bayes model
print("Training Model using Multinomial Naive Bayes algorithm...")
model = MultinomialNB() #models multinomial distribution better
model.fit(train_features, train_labels) #fit with test and train labels
print("Training completed.")

Training Model using Multinomial Naive Bayes algorithm...
Training completed.


In [59]:
#Predict labels for test data
print("Testing trained model to predict Test Data labels...")
predicted_labels = model.predict(test_features)

# Calculate accuracy
accuracy = accuracy_score(test_labels, predicted_labels)
print("Completed classification of the Test Data.")
print("Now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print(accuracy)

Testing trained model to predict Test Data labels...
Completed classification of the Test Data.
Now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9846153846153847
